# M1.3 - Lecture des données climatiques maillées de MERRA-2

*Partie de:* [**Open Climate Data**](https://github.com/OpenClimateScience/M1-Open-Climate-Data) | **Leçon précédente** | **Leçon suivante**

**Contenu :**

- [Introduction à `xarray`](#Introduction-à-xarray)
- [Utilisation d'un fichier netCDF4 téléchargé](#Utilisation-d'un-fichier-netCDF4-téléchargé)
- [Travailler avec les DataArrays de `xarray`](#Travailler-avec-les-DataArrays-de-xarray)
- [Sous-ensemble de jeux de données maillés avec `xarray`](#Sous-ensemble-de-jeux-de-données-maillés-avec-xarray)
  - [Indexation basée sur des labels](#Indexation-basée-sur-des-labels)
  - [Découpage des tableaux](#Découpage-des-tableaux)

**Maintenant que nous avons vu comment accéder à certaines données climatiques avec NASA Earthdata Search, explorons comment *utiliser* ces données en Python.**

Le fichier MERRA-2 que nous avons téléchargé a une extension de fichier `.nc4`. Cela indique qu'il s'agit d'un type de fichier appelé netCDF4, ou Network Common Data Format, version 4. Nous parlerons plus en détail du format netCDF4 plus tard. Pour l'instant, vous devez savoir que vous pouvez ouvrir ce type de fichier en utilisant une bibliothèque Python appelée `xarray`.

## Introduction à `xarray`

Nous allons utiliser un package Python appelé `xarray` pour ouvrir le fichier netCDF4 que nous avons téléchargé. `xarray` est conçu pour travailler avec des jeux de données maillés multidimensionnels.

![](./assets/xarray-dataset.png)

Comme le montre la figure ci-dessus, un seul `xarray.Dataset` peut contenir plusieurs variables comme la température, les précipitations, la latitude et la longitude. Chaque variable est stockée sous forme de tableau, plus précisément un `xarray.DataArray`. Tandis que la latitude et la longitude sont constantes dans le temps et sont donc représentées sous forme de tableaux 2D, des variables comme la température et les précipitations varient à la fois dans le temps et dans l'espace, elles peuvent donc être représentées sous forme de **cubes de données** 3D. Les axes x, y et temps (t) **axes** (également appelés **dimensions**) peuvent être utilisés pour sous-ensembles de tableaux à des périodes ou zones d'étude d'intérêt.

**Nous importons généralement `xarray` avec un nom plus court, pour le rendre plus facile à utiliser. Ci-dessous, nous importons également le module `pyplot` de `matplotlib`.**

In [ ]:
import xarray as xr
from matplotlib import pyplot

---

## Utilisation d'un fichier netCDF4 téléchargé

Nous pouvons ouvrir un fichier netCDF4 dans `xarray` en utilisant la fonction `open_dataset()`.

In [ ]:
ds = xr.open_dataset('data/MERRA2_400.statD_2d_slv_Nx.20231101.nc4')
ds

Il y a beaucoup à voir ici. Ce jeu de données a :

- **Dimensions :** Si vous travaillez avec des données cartographiques, ces données ont au moins deux dimensions (par exemple, latitude et longitude). Si la carte a été générée à partir de données satellitaires et que le satellite fournit de nouvelles observations chaque jour, nous pouvons introduire une troisième dimension, le temps. Ces dimensions décrivent la forme d'un **cube de données** avec trois **axes :** longitude ("lon"), latitude ("lat") et temps.
- **Coordonnées :** Similaires aux dimensions, les coordonnées sont les distances le long de chaque axe. Pour la longitude et la latitude, ce sont les coordonnées du centre de chaque pixel.
- **Variables de données :** Un fichier netCDF4 peut contenir différentes variables qui sont cartographiées sur la même grille. Par exemple, vous pouvez avoir à la fois la température minimale et maximale quotidienne dans le même fichier.
- **Index :** Ceux-ci sont comme les coordonnées et les dimensions, donc nous n'avons pas besoin de nous en préoccuper pour l'instant.
- **Attributs :** En plus des valeurs de données cartographiées, un fichier netCDF4 peut contenir des **métadonnées** pour aider les utilisateurs à comprendre les données. Les métadonnées sont enregistrées sous forme d'attributs et décrivent des éléments tels que la version du logiciel utilisée pour créer les données ou l'auteur des données.

Les variables dans un `xarray.Dataset` peuvent être accédées comme les clés d'un dictionnaire Python :

In [ ]:
ds['T2MMIN']

Et aussi comme une propriété :

In [ ]:
ds.T2MMIN

Chaque `Dataset` a un tableau sous-jacent. Le jeu de données `"T2MMIN"` est un tableau tridimensionnel ; nous pouvons vérifier le nom et le nombre de dimensions en accédant à la propriété `dims` :

In [ ]:
ds['T2MMIN'].dims

Cela indique que les valeurs de `"T2MMIN"` varient en fonction du temps et de deux dimensions spatiales (latitude et longitude). Même si ce jeu de données représente un seul point dans le temps, il existe toujours une dimension temporelle car le granule que nous avons téléchargé fait partie d'une série, chacun représentant une étape temporelle différente.

**L'une des particularités des fichiers netCDF4 est qu'ils peuvent stocker à la fois des données et des métadonnées, ou attributs.**

In [ ]:
ds['T2MMIN'].attrs

Les attributs peuvent stocker des informations essentielles sur les données. Par exemple, il serait difficile d'utiliser des données de température sans connaître les unités correctes.

`xarray` propose des outils pratiques intégrés pour analyser nos données, comme la possibilité de tracer des jeux de données.

In [ ]:
ds['T2MMIN'].plot()

Les tableaux de données sous-jacents sont simplement des tableaux NumPy, donc si nous voulons travailler avec un tableau NumPy à la place...

In [ ]:
ds['T2MMIN'].data

Encore une fois, le premier axe du tableau contient un seul élément, `(1)`, car ce fichier représente un seul point dans le temps.

In [ ]:
ds['T2MMIN'].data.shape

---

## Travailler avec les DataArrays de `xarray`

In [ ]:
tmin = ds['T2MMIN']
tmin.attrs

Comme avec les tableaux NumPy, nous pouvons traiter un DataArray comme un nombre, ce qui rend les transformations mathématiques de nos données faciles. Par exemple, nous pourrions vouloir convertir nos températures minimales de degrés K en degrés C.

In [ ]:
# Convertir les températures de degrés K à degrés C
tmin_c = tmin - 273.15

Une chose à savoir est que lorsque nous faisons ce type d'opération, nous perdons les attributs du DataArray d'origine. Cela s'explique par le fait que les anciens attributs peuvent ne plus s'appliquer ; en fait, nous savons déjà que les "unités" du DataArray d'origine (degrés K) ne sont plus exactes.

In [ ]:
tmin_c.attrs

Nous pouvons attribuer des attributs à tout moment, en utilisant une syntaxe de dictionnaire Python.

In [ ]:
tmin_c.attrs['units'] = 'degrés K'
tmin_c.attrs

Quel est l'intérêt d'attribuer de nouveaux attributs ? Vous devriez faire cela chaque fois que vous allez enregistrer un Dataset ou un DataArray dans un fichier de sortie et le partager avec quelqu'un. Les Datasets et DataArrays ont une méthode, `to_netcdf()`, qui vous permet de faire exactement cela.

In [ ]:
tmin_c.to_netcdf('exemple.nc4')

---

## Sous-ensemble de jeux de données maillés avec `xarray`

Parce que nous avons téléchargé des données moyennes quotidiennes de MERRA-2, il n'y a qu'une seule grille 2D de températures dans ce jeu de données. Cela signifie que notre dimension temporelle a une longueur de un.

In [ ]:
tmin.shape

Si nous devions sous-ensemble notre jeu de données à un moment spécifique, nous pourrions utiliser des indices numériques, tout comme avec un tableau NumPy.

In [ ]:
tmin[0]

De même, si nous voulions obtenir une série chronologique de valeurs à des coordonnées spécifiques (ligne-colonne), nous pourrions écrire :

In [ ]:
# Obtenez toutes les valeurs sur l'axe temporel pour la position : ligne 50, colonne 100
tmin[:,50,100]

**Mais nous ne connaissons souvent pas la position exacte des lignes-colonnes des endroits qui nous intéressent. Comment pouvons-nous sélectionner des valeurs basées sur la longitude et la latitude, à la place ?**

Nous avons de la chance, car notre DataArray a des coordonnées qui décrivent où chaque valeur de données est située.

In [ ]:
ds.coords

### Indexation basée sur des labels

Parce que `xarray` sait ce que représente chaque dimension et possède des coordonnées, `coords`, pour chacune des dimensions, il est possible de demander les valeurs de données à des coordonnées spécifiques. Par exemple, nous pouvons nous référer à une étape temporelle spécifique :

In [ ]:
ds['T2MMIN'].loc['2023-11-01']

Et même la valeur à une latitude et longitude spécifiques :

In [ ]:
ds['T2MMIN'].loc['2023-11-01',45.5,-80]

Remarquez que, lors de l'utilisation de `ds['T2MMIN'].loc`, nous utilisons des crochets, comme si nous indexions un dictionnaire ou un tableau NumPy. Nous devons également fournir les labels pour chaque coordonnée dans l'ordre où ils sont attendus, c'est-à-dire l'ordre qui apparaît dans :

In [ ]:
ds['T2MMIN'].dims

Pour être plus explicite sur les dimensions que nous indexons, nous pouvons *sélectionner* la ou les valeurs à certaines coordonnées en utilisant la fonction `sel()`.

In [ ]:
# Température minimale à 2 mètres au pôle Sud
ds['T2MMIN'].sel(lat = -90, lon = -180)

Remarque : Dans un autre jeu de données, les coordonnées de latitude et de longitude peuvent avoir des noms différents !

In [ ]:
ds['T2MMIN'].sel(lat = -90, lon = -180).values

Quelle est la température minimale à Alger ?

In [ ]:
ds['T2MMIN'].sel(lat = 36.754, lon = 3.059)

Que s'est-il passé ? Si nous examinons les coordonnées de notre jeu de données, nous verrons qu'il n'y a pas de correspondance exacte pour la paire longitude-latitude que nous avons fournie ; elles ne sont disponibles qu'à des intervalles régulièrement espacés de 0,25 ou 0,5 degré.

In [ ]:
ds['lon'].values[0:10]

In [ ]:
ds['lat'].values[0:10]

Notez que nous devons spécifier une `méthode` ici, car les coordonnées d'Alger ne correspondent pas exactement aux coordonnées du centre de chaque cellule de la grille ; c'est-à-dire que nous devons demander une interpolation par voisin le plus proche.

In [ ]:
ds['T2MMIN'].sel(lat = 36.754, lon = 3.059, method = 'nearest').values

Une autre façon d'obtenir la réponse que nous voulons est d'utiliser la fonction `interp()`. **Remarquez que la réponse que nous obtenons est légèrement différente de celle obtenue ci-dessus.** C'est parce que la réponse précédente utilisait une *interpolation par voisin le plus proche*, mais la fonction `interp()` utilise par défaut une *interpolation linéaire*. Il existe de nombreuses autres options pour l'interpolation avec la fonction `interp()`.

In [ ]:
ds['T2MMIN'].interp(lat = 36.754, lon = 3.059)